# import dependancies

In [1]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd

# database connection

In [2]:
conn = psycopg2.connect(database="postgres", user="postgres", password="postgres", host="localhost", port = "5432")
cur = conn.cursor()

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?


## import data function

In [ ]:
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()
    return query.format(*tuples)

# load data

In [3]:
try:
    df = pd.read_csv('https://s3.amazonaws.com/coderbyteprojectattachments/simcel-6pk70-1jk5iqdp-train_v9rqX0R.csv')
except:
    df = pd.read_csv('simcel.csv')

# process data

In [4]:
item = df[['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Type']]
item = item.sort_values(by=['Item_Identifier', 'Item_Weight'], ascending=True)
item.ffill(inplace=True)
# item['Item_Fat_Content'].unique()
# item['Item_Type'].unique()
item['Item_Fat_Content'] = item['Item_Fat_Content'].replace('LF', 'Low Fat').replace('low fat', 'Low Fat').replace('reg', 'Regular')
item.drop_duplicates(inplace=True)

In [5]:
outlet = df[['Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']]
outlet.drop_duplicates(inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_18956\2712254571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlet.drop_duplicates(inplace=True)


In [6]:
sale_report = df[['Item_Identifier', 'Outlet_Identifier', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales']]

# create tables and import data to database

## item table

In [ ]:
sql = """
CREATE TABLE item (
    item_identifier CHAR(5) PRIMARY KEY,
    item_weight REAL,
    item_fat_content VARCHAR(10),
    item_type VARCHAR(30)
);
"""
cur.execute(sql)

In [ ]:
execute_values(conn, item, 'item')

execute_values() done


'INSERT INTO item(Item_Identifier,Item_Weight,Item_Fat_Content,Item_Type) VALUES %s'

In [18]:
# tuples = [tuple(x) for x in item.to_numpy()]
# for data in tuples:
#     print("""INSERT INTO item (item_identifier, item_weight, item_fat_content, item_type) VALUES ('{}', {}, '{}', '{}');""".format(*data))

## store table

In [ ]:
sql = """
CREATE TABLE outlet (
    outlet_identifier CHAR(6) PRIMARY KEY,
    outlet_establishment_year SMALLINT,
    outlet_size VARCHAR(10),
    outlet_location_Type CHAR(6),
    outlet_type VARCHAR(20)
);
"""
cur.execute(sql)

In [ ]:
execute_values(conn, outlet, 'outlet')

execute_values() done


'INSERT INTO outlet(Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type) VALUES %s'

In [17]:
# tuples = [tuple(x) for x in outlet.to_numpy()]
# for data in tuples:
#     print("""INSERT INTO outlet (outlet_identifier, outlet_establishment_year, outlet_size, outlet_location_Type, outlet_type) VALUES ('{}', {}, '{}', '{}', '{}');""".format(*data))

## sale_report table

In [ ]:
sql = """
CREATE TABLE sale_report (
    id SERIAL PRIMARY KEY,
    item_identifier CHAR(5) REFERENCES item(Item_Identifier),
    outlet_identifier CHAR(6) REFERENCES outlet(Outlet_Identifier),
    item_visibility REAL,
    item_mrp REAL,
    item_outlet_sales REAL
);
"""
cur.execute(sql)

In [7]:
sale_report['id'] = sale_report.index
execute_values(conn, sale_report, 'sale_report')

C:\Users\user\AppData\Local\Temp\ipykernel_18956\360566809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sale_report['id'] = sale_report.index


In [16]:
# tuples = [tuple(x) for x in sale_report.to_numpy()]
# for data in tuples:
#     print("""INSERT INTO sale_report (item_identifier, outlet_identifier, item_visibility, item_mrp, item_outlet_sales, id) VALUES ('{}', '{}', '{}', {}, {}, {});""".format(*data))